In [ ]:
import ultralytics
ultralytics.checks()

## Custom Training

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11x.pt")  # load a pretrained model (recommended for training)

In [ ]:
# Train the model
results = model.train(task="detect", data="data.yaml", epochs=24, imgsz=640, batch=-1, device=0, plots=True)

In [ ]:
# Validate the model
metrics = model.val(task="detect", data="data.yaml", imgsz=640, device=0, plots=True)  # no arguments needed, dataset and settings remembered, but in case of interruptions...

In [ ]:
# Run batched inference on a list of images
foto = model("porcino.jpg")  # return a list of Results objects

# Process results list
for funghi in foto:
    boxes = funghi.boxes  # Boxes object for bounding box outputs
    masks = funghi.masks  # Masks object for segmentation masks outputs
    keypoints = funghi.keypoints  # Keypoints object for pose outputs
    probs = funghi.probs  # Probs object for classification outputs
    obb = funghi.obb  # Oriented boxes object for OBB outputs
    funghi.show()  # display to screen
    funghi.save(filename="result2.jpg")  # save to disk

In [ ]:
%%capture
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from ultralytics import YOLO

# Carica il modello YOLO11x
model = YOLO("runs/detect/train/weights/best.pt")

# Percorsi delle cartelle
image_folder = "test/images"
label_folder = "test/labels"

# Funzione per leggere le etichette
def read_labels(label_path):
    with open(label_path, 'r') as file:
        lines = file.readlines()
    labels = []
    for line in lines:
        class_id, x_center, y_center, width, height = map(float, line.strip().split())
        labels.append((class_id, x_center, y_center, width, height))
    return labels

# Esegui inferenza e raccogli le predizioni e le etichette
y_true = []
y_scores = []

for image_file in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_file)
    label_path = os.path.join(label_folder, os.path.splitext(image_file)[0] + '.txt')
    
    if not os.path.exists(label_path):
        continue

    labels = read_labels(label_path)
    
    results = model(image_path)
    
    true_count = len(labels)  # Numero reale di funghi
    pred_count = 0
    
    # Raccolta delle predizioni e delle etichette
    for funghi in results:
        for box in funghi.boxes:
            score = box.conf[0].cpu().numpy()  # Punteggio di confidenza della predizione
            y_scores.append(score)
            pred_count += 1 if score >= 0.25 else 0

    y_true.extend([1] * true_count)  # Funghi veri (1)
    y_true.extend([0] * (pred_count - true_count if pred_count > true_count else 0))  # Background falso (0)
    
    y_scores.extend([0] * (true_count - pred_count if true_count > pred_count else 0))  # Funghi mancanti

# Calcola la curva ROC
fpr, tpr, thresholds = roc_curve(y_true, y_scores)
roc_auc = auc(fpr, tpr)

# Calcola Youden's J statistic (tpr - fpr) per ogni valore di soglia
youden = tpr - fpr

# Trova la soglia ottimale che massimizza la differenza tra tpr e fpr
optimal_threshold = thresholds[np.argmax(youden)]

# Traccia la curva ROC
plt.figure()
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], 'k--')  # Diagonal reference line

# Annota la soglia ottimale con un marker a stella
plt.scatter(fpr[np.argmax(youden)], tpr[np.argmax(youden)], marker='*', color='black', s=100, label=f'Optimal Threshold = {optimal_threshold:.4f}', zorder=5)

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.savefig('roc.png', bbox_inches='tight')
plt.show()

print(f'Optimal Threshold: {optimal_threshold}')

In [ ]:
# Validate the model
metrics = model.val(task="detect", data="data.yaml", imgsz=640, device=0, plots=True, conf=optimal_threshold)  # using the optimal threshold

In [ ]:
# Run batched inference on a list of images
foto = model(["4 (1).jpg", "309382.jpg", "53724.jpg", "porcino.jpg", "porci.jpg", "hard2.jpg", "ama.jpg", "boh.jpg", "pene.jpg", "litt.jpg"])#, conf=0.78)  # return a list of Results objects

# Process results list
for funghi in foto:
    boxes = funghi.boxes  # Boxes object for bounding box outputs
    probs = funghi.probs  # Probs object for classification outputs
    funghi.show()  # display to screen